# Analyze 2018 Pilot runs in Parallel

## Shell Variables
Assign the variables in this notebook.

In [ ]:
source stress_test_config.sh

### Running the full pipeline on all the samples

In [ ]:
star_func() {
    FASTQ=$1
    FASTQ_BASE=${FASTQ##*/} # strip directory from file path
    SAMPLE="${FASTQ_BASE%_R1_001.fastq.gz}" # strip .fq.gz file extension
    echo $SAMPLE
    echo $FASTQ
    echo $FASTQ_BASE
    # exit 1
    
    /usr/bin/time -v --format='(%Xtext+%Ddata %Mmax)' STAR \
    --runMode alignReads \
    --runThreadN $THREADS \
    --readFilesCommand zcat \
    --genomeDir $GENOME_DIR \
    --outSAMtype BAM SortedByCoordinate \
    --quantMode GeneCounts \
    --genomeLoad LoadAndKeep \
    --twopassMode None \
    --limitBAMsortRAM 1280000000 \
    --outFileNamePrefix ${STAR_OUT}/${SAMPLE}_ \
    --outFilterScoreMinOverLread 0 --outFilterMatchNminOverLread 0 \
    --outFilterMatchNmin 0 --outFilterMismatchNmax 2 \
    --readFilesIn $TRIMMED/${SAMPLE}.trim.fastq.gz
}
export -f star_func

STAR --genomeDir $GENOME_DIR \
    --outFileNamePrefix ${STAR_OUT}/genomeload_ \
    --genomeLoad LoadAndExit 

# parallel --jobs $MAX_JOBS star_func ::: $RAW_FASTQS/35_MA_*_L001_R1_001.fastq.gz
# parallel --jobs $MAX_JOBS star_func ::: $RAW_FASTQS/*_L001_R1_001.fastq.gz
# parallel --jobs $MAX_JOBS star_func ::: $RAW_FASTQS/[3-4]_RZ_*_L001_R1_001.fastq.gz
parallel --jobs $MAX_JOBS star_func ::: $RAW_FASTQS/4_RZ_P_S43_L003_R1_001.fastq.gz $RAW_FASTQS/3_TOT_J_S34_L003_R1_001.fastq.gz

STAR --genomeDir $GENOME_DIR \
    --outFileNamePrefix ${STAR_OUT}/genomeremove_ \
    --genomeLoad Remove 